In [2]:
!pip install -q youtube-transcript-api langchain-community langchain-openai \ faiss-cpu tiktoken python-dotenv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00


In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi,TranscriptsDisabled
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate


In [ ]:
!pip install -q youtube-transcript-api langchain-community langchain-openai faiss-cpu tiktoken python-dotenv

In [ ]:
from dotenv import load_dotenv
import os, requests
from langchain.llms import BaseLLM
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from pydantic import Field, BaseModel

# Set the environment variable before loading
# Using a standard name for consistency and clarity
os.environ["TOGETHER_AI_API_KEY"]="tgp_v1_OSpvv_Dzn_65Tk0F6nEMJAcf8eyqMLgiVBAhqy8djvM"

# Load .env (if you were using a .env file, this would load it)
# Since we're setting it directly, load_dotenv might not be strictly necessary here,
# but keep it if you plan to use a .env file later.
load_dotenv()

# Load the API key from the environment variable
API_KEY = os.getenv("TOGETHER_AI_API_KEY")

# Add a check to ensure the API key was loaded
if not API_KEY:
    raise ValueError("TOGETHER_AI_API_KEY environment variable not set.")


from langchain_core.outputs import LLMResult, Generation
from typing import List, Optional, Any

class TogetherAIWrapper(BaseLLM):
    api_key: str = Field(..., description="API key for Together.ai")

    def __init__(self, api_key: str,**kwargs: Any):
      super().__init__(api_key=api_key,**kwargs)

    def _llm_type(self) -> str:
        return "together_ai"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        url = "https://api.together.xyz/inference"
        data = {
            "model": "mistralai/Mistral-7B-Instruct-v0.1",
            "prompt": prompt,
            "max_tokens": 300,
            "temperature": 0.7,
            "top_p": 0.9
        }
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }

        response = requests.post(url, headers=headers, json=data)
        if response.status_code == 200:
            # Assuming the API returns the text directly in the "output" key
            # If the structure is ['output']['choices'][0]['text'], adjust here
            response_data = response.json()
            # Let's try to get the text in a way that's compatible with the original working version
            # and the structure needed for Generation.
            # The original working version used ['output']['choices'][0]['text']
            try:
                return response_data['output']['choices'][0]['text']
            except (KeyError, IndexError) as e:
                raise Exception(f"Error parsing Together.ai API response: {e}. Response: {response_data}")

        else:
            raise Exception(f"Error: {response.status_code}, {response.text}")

    def _generate(
        self,
        prompts: List[str],
        stop: Optional[List[str]] = None,
        **kwargs: Any,
    ) -> LLMResult:
        generations = []
        for prompt in prompts:
            text = self._call(prompt, stop=stop)
            # Create a Generation object with the retrieved text
            generations.append([Generation(text=text)])
        # LLMResult expects a list of lists of Generation objects
        return LLMResult(generations=generations)





ML can improve healthcare by predicting and diagnosing diseases, developing personalized treatment plans, analyzing medical images, improving patient outcomes and reducing healthcare costs.

Explanation: 

1. Predicting and diagnosing diseases: ML algorithms can analyze vast amounts of patient data to identify patterns and risk factors for various diseases. This can help healthcare providers to diagnose diseases early and develop effective treatment plans.
2. Developing personalized treatment plans: ML algorithms can analyze patient data to identify the most effective treatment options for individual patients. This can help healthcare providers to develop personalized treatment plans that are tailored to each patient's unique needs and circumstances.
3. Analyzing medical images: ML algorithms can analyze medical images such as X-rays, CT scans, and MRIs to identify abnormalities and assist healthcare providers in diagnosing diseases.
4. Improving patient outcomes: ML can be used to an

In [1]:
#Step 1A --Indexing (Document INgestion)

video_id = "Gfr50f6ZBvo"
try:
  transcript_list = YouTubeTranscriptApi.get_transcript(video_id,languages=["en"])
  transcript = " ".join(chunk["text"]for chunk in transcript_list)
  print(transcript)
except TranscriptsDisabled:
  print("No caption available")


NameError: name 'TranscriptsDisabled' is not defined

In [ ]:
#Step 1b -- (text splitting)
splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks = splitter.create_documents([transcript])


In [ ]:
len(chunks)

168

Step 1c&d ---Creating embedding and storing in vectorStorees


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en")
vector_store = FAISS.from_documents(chunks,embeddings)



ModuleNotFoundError: Module langchain_community.embeddings not found. Please install langchain-community to access this module. You can install it using `pip install -U langchain-community`

Step 2 -- Retrieval


In [ ]:
retriever = vector_store.as_retriever(search_type="similarity",search_kwargs={"k":4})

In [ ]:
llm = TogetherAIWrapper("tgp_v1_OSpvv_Dzn_65Tk0F6nEMJAcf8eyqMLgiVBAhqy8djvM")


In [ ]:
promt = PromptTemplate(template="""
you are a helpful assistant.
Answer Only from provided transcript context
If the context is insufficient. just say you don't know ,
{context}
Question:{question}""",input_variables=["context","question"])

In [ ]:
question= "is the topic of alien discussed in this video? if yes then what was discussed"
retrived_doc = retriever.invoke(question)

In [ ]:
content_text = "\n\n".join(doc.page_content for doc in retrived_doc)

In [ ]:
final_prompt = promt.invoke({"context":content_text,"question":question})

In [ ]:
answer = llm.invoke(final_prompt)
print(answer)

?

Answer: The topic of aliens is discussed in this video, but it is not the main topic. The main topic is the relationship between artificial intelligence and creativity. The discussion about aliens is brought up in the context of how AI systems might be able to create something original, and the idea that AI systems might be able to surpass human creativity in certain areas. The discussion about aliens is used as an example of how AI systems might be able to create something original, and the idea that AI systems might be able to surpass human creativity in certain areas.


Building a Chain ....

In [ ]:
from langchain_core.runnables import RunnableParallel,RunnablePassthrough,RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [ ]:
def format_doc(retrived_doc):
  content_text="\n\n".join(doc.page_content for doc in retrived_doc)
  return content_text

In [ ]:
parallel_chain = RunnableParallel({
    "context":retriever | RunnableLambda(format_doc),
    "question":RunnablePassthrough()
})


In [ ]:
parser = StrOutputParser()


In [ ]:
main_chain = parallel_chain|promt|llm|parser

In [ ]:
main_chain.invoke("can you summarize this video")

' in a few sentences?\n\nAnswer:\nThe video discusses the evolution of artificial intelligence, specifically in the field of game playing. The development of AlphaGo and AlphaZero, and their ability to play against themselves and generalize to any two player game, is highlighted. The video also discusses the importance of clear and simple explanations as a sign of understanding and intelligence.'